<a href="https://colab.research.google.com/github/Mohit54/GTR/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training Demo
This is a simple example for training the SimSwap 224*224 with VGGFace2-224.

Code path: https://github.com/neuralchen/SimSwap
If you like the SimSwap project, please star it!
Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sat Aug 30 11:02:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Installation
All file changes made by this notebook are temporary. You can try to mount your own google drive to store files if you want.

#Get Scripts

In [7]:
!git clone https://github.com/neuralchen/SimSwap /content/drive/MyDrive/LRisk_Sln/SimSwap

Cloning into '/content/drive/MyDrive/LRisk_Sln/SimSwap'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (508/508), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 1141 (delta 419), reused 382 (delta 382), pack-reused 633 (from 1)
Receiving objects: 100% (1141/1141), 211.46 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (601/601), done.



In [8]:
%cd /content/drive/MyDrive/LRisk_Sln/SimSwap

/content/drive/MyDrive/LRisk_Sln/SimSwap


In [9]:
!git pull

Updating files: 100% (147/147), done.


In [11]:
!cwd

/bin/bash: line 1: cwd: command not found


In [12]:
import os
print(os.getcwd())

/content/drive/MyDrive/LRisk_Sln/SimSwap


# Install Blocks

In [13]:
!pip install googledrivedownloader
!pip install timm==0.5.4
!wget -P SimSwap/arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar

--2025-08-30 11:12:33--  https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/374891081/f01468b3-446b-4867-8c78-6d496183f9e6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-30T12%3A07%3A12Z&rscd=attachment%3B+filename%3Darcface_checkpoint.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-30T11%3A06%3A53Z&ske=2025-08-30T12%3A07%3A12Z&sks=b&skv=2018-11-09&sig=CdyDT2ZKp%2F3BUQrAvLWv2eW0b58VvdWgLHcdL2HwqRo%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NjU1MjY1MywibmJmIjoxNzU2NTUyMzUzLCJwYXRoIjoicmVsZWFzZWFzc2V0cH

#Download the Training Dataset
We employ the cropped VGGFace2-224 dataset for this toy training demo.

You can download the dataset from our google driver https://drive.google.com/file/d/19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc/view?usp=sharing

***Please check the dataset in dir /content/TrainingData***

***If dataset already exists in /content/TrainingData, please do not run blow scripts!***


In [21]:
%mkdir /content/drive/MyDrive/LRisk_Sln/TrainingData
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc" -O /content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar && rm -rf /tmp/cookies.txt
%cd /content/
!tar -xzvf /content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar -C /content/TrainingData
!rm /content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar

--2025-08-30 11:24:14--  https://docs.google.com/uc?export=download&confirm=&id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc
Resolving docs.google.com (docs.google.com)... 172.253.118.101, 172.253.118.113, 172.253.118.100, ...
Connecting to docs.google.com (docs.google.com)|172.253.118.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc&export=download [following]
--2025-08-30 11:24:16--  https://drive.usercontent.google.com/download?id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.24.132, 2404:6800:4003:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.24.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2445 (2.4K) [text/html]
Saving to: ‘/content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar’

/content/d

In [22]:
!pip install gdown

# Create folder
%mkdir -p /content/drive/MyDrive/LRisk_Sln/TrainingData

# Download dataset from Google Drive using gdown
!gdown --id 19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc -O /content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar

# Extract
!mkdir -p /content/TrainingData
!tar -xvf /content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar -C /content/TrainingData

# Remove tar file
!rm /content/drive/MyDrive/LRisk_Sln/TrainingData/vggface2_crop_arcfacealign_224.tar


Streaming output truncated to the last 5000 lines.
vggface2_crop_arcfacealign_224/n007577/0363_01.jpg
vggface2_crop_arcfacealign_224/n007577/0114_01.jpg
vggface2_crop_arcfacealign_224/n007577/0173_01.jpg
vggface2_crop_arcfacealign_224/n007577/0271_01.jpg
vggface2_crop_arcfacealign_224/n007577/0177_02.jpg
vggface2_crop_arcfacealign_224/n007577/0111_01.jpg
vggface2_crop_arcfacealign_224/n007577/0146_01.jpg
vggface2_crop_arcfacealign_224/n007577/0291_01.jpg
vggface2_crop_arcfacealign_224/n007577/0308_01.jpg
vggface2_crop_arcfacealign_224/n007577/0161_01.jpg
vggface2_crop_arcfacealign_224/n007577/0100_01.jpg
vggface2_crop_arcfacealign_224/n007577/0296_01.jpg
vggface2_crop_arcfacealign_224/n007577/0203_01.jpg
vggface2_crop_arcfacealign_224/n007577/0269_01.jpg
vggface2_crop_arcfacealign_224/n007577/0210_01.jpg
vggface2_crop_arcfacealign_224/n007577/0318_01.jpg
vggface2_crop_arcfacealign_224/n007577/0417_01.jpg
vggface2_crop_arcfacealign_224/n007577/0165_01.jpg
vggface2_crop_arcfacealign_224/

In [42]:
import os
from mtcnn import MTCNN
import cv2
from tqdm import tqdm

detector = MTCNN()

SRC = "/content/drive/MyDrive/LRisk_Sln/dataset/lfw_funneled"   # path to LFW dataset
OUT = "/content/drive/MyDrive/LRisk_Sln/224_x_224_faces"
os.makedirs(OUT, exist_ok=True)

def align_and_save(src_path, out_dir, out_name, size=(256,256), margin_ratio=0.2):
    img = cv2.imread(src_path)
    if img is None:
        return False

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    dets = detector.detect_faces(img_rgb)
    if len(dets) == 0:
        return False

    x, y, w, h = dets[0]['box']  # x,y,w,h
    h_img, w_img = img.shape[:2]

    # margin proportional to face size
    margin = int(margin_ratio * max(w, h))

    x1 = max(0, x - margin)
    y1 = max(0, y - margin)
    x2 = min(w_img, x + w + margin)
    y2 = min(h_img, y + h + margin)

    face = img[y1:y2, x1:x2]
    if face.size == 0:
        return False

    face = cv2.resize(face, size)

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, out_name)
    cv2.imwrite(out_path, face)  # save directly in BGR
    return True

# Walk LFW folders
persons = sorted([p for p in os.listdir(SRC) if os.path.isdir(os.path.join(SRC, p))])

count = 0
for person in tqdm(persons):
    in_dir = os.path.join(SRC, person)
    out_person_dir = os.path.join(OUT, person)
    for i, imgf in enumerate(sorted(os.listdir(in_dir))):
        if not imgf.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        src_path = os.path.join(in_dir, imgf)
        out_name = f"{person}_{i+1:04d}.jpg"
        dst_path = os.path.join(out_person_dir, out_name)

        if os.path.exists(dst_path):
            continue

        ok = align_and_save(src_path, out_person_dir, out_name)
        if ok:
            count += 1

print("Aligned faces:", count)


100%|██████████| 5749/5749 [3:23:53<00:00,  2.13s/it]

Aligned faces: 13232


In [41]:
!pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00


#Trainig
Batch size must larger than 1!

In [43]:
%cd /content/SimSwap
!ls
!python /content/drive/MyDrive/LRisk_Sln/SimSwap/train.py --name simswap224_test_224 --gpu_ids 0 --dataset /content/drive/MyDrive/LRisk_Sln/224_x_224_faces --Gdeep False

/content/SimSwap
 arcface_model	       predict.py
 checkpoints	       README.md
 cog.yaml	      'SimSwap colab.ipynb'
 crop_224	       simswaplogo
 data		       test_one_image.py
 demo_file	       test_video_swapmulti.py
 docs		       test_video_swap_multispecific.py
 download-weights.sh   test_video_swapsingle.py
 insightface_func      test_video_swapspecific.py
 LICENSE	       test_wholeimage_swapmulti.py
 models		       test_wholeimage_swap_multispecific.py
 MultiSpecific.ipynb   test_wholeimage_swapsingle.py
 options	       test_wholeimage_swapspecific.py
 output		       train.ipynb
 parsing_model	       train.py
 pg_modules	       util
2025-08-30 18:50:28.643447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756579828.665088  127711 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN whe

In [44]:
!python /content/drive/MyDrive/LRisk_Sln/SimSwap/train.py \
    --name /content/drive/MyDrive/LRisk_Sln/simswap224_test_224 \
    --gpu_ids 0 \
    --dataset /content/drive/MyDrive/LRisk_Sln/224_x_224_faces \
    --Gdeep False \
    --batchSize 32 \
    --checkpoints_dir /content/drive/MyDrive/LRisk_Sln/checkpoints_224


2025-08-30 18:52:05.163423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756579925.184640  128203 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756579925.191133  128203 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756579925.207474  128203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756579925.207499  128203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756579925.207502  128203 computation_placer.cc:177] computation placer alr

In [48]:

%cd /content/drive/MyDrive/LRisk_Sln/SimSwap

!python test_one_image.py \
  --name simswap224_test_224 \
  --Arc_path /content/drive/MyDrive/LRisk_Sln/SimSwap/SimSwap/arcface_model/arcface_checkpoint.tar \
  --pic_a_path /content/drive/MyDrive/LRisk_Sln/Aaron_Eckhart_0001.jpg \
  --pic_b_path /content/drive/MyDrive/LRisk_Sln/Adam_Kennedy_0001.jpg \
  --which_epoch 10000 \
  --checkpoints_dir /content/drive/MyDrive/LRisk_Sln

/content/drive/MyDrive/LRisk_Sln/SimSwap
------------ Options -------------
Arc_path: /content/drive/MyDrive/LRisk_Sln/SimSwap/SimSwap/arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: /content/drive/MyDrive/LRisk_Sln
cluster_path: features_clustered_010.npy
crop_size: 512
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: simswap224_test_224
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: False
norm: batch
norm_G: spectral

In [35]:
%cd /content/drive/MyDrive/LRisk_Sln/SimSwap

!python test_one_image.py \
  --name simswap224_test_new \
  --Arc_path /content/drive/MyDrive/LRisk_Sln/SimSwap/SimSwap/arcface_model/arcface_checkpoint.tar \
  --pic_a_path /content/drive/MyDrive/LRisk_Sln/Bale.png \
  --pic_b_path /content/drive/MyDrive/LRisk_Sln/mohit54.jpg \
  --which_epoch 10000 \
  --load_size 256 \
  --crop_size 256 \
  --checkpoints_dir /content/drive/MyDrive/LRisk_Sln/

/content/drive/MyDrive/LRisk_Sln/SimSwap
usage: test_one_image.py [-h] [--name NAME] [--gpu_ids GPU_IDS]
                         [--checkpoints_dir CHECKPOINTS_DIR] [--norm NORM]
                         [--use_dropout] [--data_type {8,16,32}] [--verbose]
                         [--fp16] [--local_rank LOCAL_RANK]
                         [--isTrain ISTRAIN] [--batchSize BATCHSIZE]
                         [--loadSize LOADSIZE] [--fineSize FINESIZE]
                         [--label_nc LABEL_NC] [--input_nc INPUT_NC]
                         [--output_nc OUTPUT_NC] [--dataroot DATAROOT]
                         [--resize_or_crop RESIZE_OR_CROP] [--serial_batches]
                         [--no_flip] [--nThreads NTHREADS]
                         [--max_dataset_size MAX_DATASET_SIZE]
                         [--display_winsize DISPLAY_WINSIZE] [--tf_log]
                         [--netG NETG] [--latent_size LATENT_SIZE] [--ngf NGF]
                         [--n_downsample_global N_DOWN

In [39]:
%cd /content/drive/MyDrive/LRisk_Sln/SimSwap

!python test_one_image.py \
  --name simswap224_test_new \
  --Arc_path /content/drive/MyDrive/LRisk_Sln/SimSwap/SimSwap/arcface_model/arcface_checkpoint.tar \
  --pic_a_path /content/drive/MyDrive/LRisk_Sln/Aaron_Eckhart_0001.jpg \
  --pic_b_path /content/drive/MyDrive/LRisk_Sln/Adam_Kennedy_0001.jpg\
  --which_epoch 10000 \
  --checkpoints_dir /content/drive/MyDrive/LRisk_Sln/ \
  --image_size 256 \
  --no_simswaplogo \
  --use_mask


/content/drive/MyDrive/LRisk_Sln/SimSwap
------------ Options -------------
Arc_path: /content/drive/MyDrive/LRisk_Sln/SimSwap/SimSwap/arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: /content/drive/MyDrive/LRisk_Sln/
cluster_path: features_clustered_010.npy
crop_size: 512
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 256
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: simswap224_test_new
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: True
norm: batch
norm_G: spectral

In [34]:
!python test_one_image.py \
    --name simswap256 \
    --Arc_path ./arcface_model/arcface_checkpoint.tar \
    --pic_a_path ./demo_file/source.jpg \
    --pic_b_path ./demo_file/target.jpg \
    --isTrain false \
    --no_simswaplogo \
    --use_mask \
    --load_size 256 \
    --crop_size 256 \
    --which_epoch 10000 \
    --G_path /content/drive/MyDrive/LRisk_Sln/simswap224_test_new/10000_net_G.pth


usage: test_one_image.py [-h] [--name NAME] [--gpu_ids GPU_IDS]
                         [--checkpoints_dir CHECKPOINTS_DIR] [--norm NORM]
                         [--use_dropout] [--data_type {8,16,32}] [--verbose]
                         [--fp16] [--local_rank LOCAL_RANK]
                         [--isTrain ISTRAIN] [--batchSize BATCHSIZE]
                         [--loadSize LOADSIZE] [--fineSize FINESIZE]
                         [--label_nc LABEL_NC] [--input_nc INPUT_NC]
                         [--output_nc OUTPUT_NC] [--dataroot DATAROOT]
                         [--resize_or_crop RESIZE_OR_CROP] [--serial_batches]
                         [--no_flip] [--nThreads NTHREADS]
                         [--max_dataset_size MAX_DATASET_SIZE]
                         [--display_winsize DISPLAY_WINSIZE] [--tf_log]
                         [--netG NETG] [--latent_size LATENT_SIZE] [--ngf NGF]
                         [--n_downsample_global N_DOWNSAMPLE_GLOBAL]
                         [